## Image classification

Za domácí úkol budete mít vytvořit si vlastní libovolný dataset na image classfication task, abyste si vyzkoušeli, že vyrábět data není úplně snadné. Dataset by měl mít aspoň 5 různých tříd ke klasifikaci a ke každé nějaké rozumné desítky obrázků.

Až budete mít data, tak si na nich zkuste image classification task. Jak už jsem zmínila výše, trénovat celou konvoluční síť od začátku by bylo početně náročné, proto budete mít za úkol si vybrat 3 z už predtrénovaných modelů například [odsud](https://www.pyimagesearch.com/2017/03/20/imagenet-vggnet-resnet-inception-xception-keras/) a přetrénovat je tak, aby uměly klasifikovat vámi vytvořená data.

Můžete si na vašich datech i zkusit natrénovat podobnou (klidně i stejnou) jednoduchou konvoluční síť, kterou jsme používali před na MNISTu výše a schválně se podívat, jak moc bude špatná oproti těm už předtrénovaným.

Své řešení mi pošlete emailem do stanoveného deadline. Pro sepsání popisu vašeho datasetu, zvolených modelů a rozboru výsledků použijte [tento template](https://github.com/kackamac/Prirodou-inspirovane-algoritmy/blob/master/08_konvoluce/DU2_konvolucni_site.pdf). Řešení by mělo obsahovat:
1. slovní popis vašich dat (jaké máte labely, kolik každý obsahuje obrázků, velikost obrázků, zdůvodnění volby datasetu)
2. váš kód
3. slovní popis výsledků a porovnání modelů (jaké jste použili, jak byl který úspěšný (accuraccy všech modelů na testovací množině, sepsaná třeba v nějaké pěkné tabulce) a který byl nejlepší)
4. vámi vytvořený předzpracovaný dataset (obrázky by měly být malé, takže by neměl být problém s posláním)

Pár tipů
- bylo by dobré mít u všech tříd stejný počet obrázků
- obrázky bude potřeba převést do nějakého jednotného rozumně malého rozměru (například 128 x 128 pixelů)
- když budete používat předtrénované sítě, tak ty vaše obrázky by měly být stejně velké jako ty, na kterých byla trénovaná ta  síť původně
- zároveň bude potřeba je rozdělit na dvě disjunktní množiny -- trénovací a testovací (trénovací na přetrénovaní modelů a testovací na jejich vyhodnocení)
- model se dá přetrénovat například tak, že si nahradíte poslední nebo několik posledních vrstev vlastními, které budete přetrénovávat podle vašich dat
- pokud se vám model nedaří přetrénovat, aby dosahoval nějakých rozumných výsledků, zkuste použít nějakou augmentaci dat a trénovat na větších datech


In [31]:
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.keras as keras
%matplotlib inline
from tensorflow.keras.datasets import cifar10, mnist, fashion_mnist
from tensorflow.keras.layers import InputLayer, Conv2D, MaxPool2D, Flatten, Dense, Activation, Resizing, Rescaling, RandomFlip, RandomRotation
from tensorflow.keras.activations import linear, relu
from tensorflow.nn import softmax
from tensorflow.keras.utils import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input, decode_predictions

In [25]:
batch_size = 32

train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset",
    validation_split=0.3,
    subset="both",
    seed=42,
    batch_size=batch_size
)

Found 38 files belonging to 2 classes.
Using 27 files for training.
Using 11 files for validation.


In [28]:
IMG_SIZE = 224

resize_and_rescale = tf.keras.Sequential([
  Resizing(IMG_SIZE, IMG_SIZE),
  Rescaling(1./255)
])

In [32]:
data_augmentation = tf.keras.Sequential([
  RandomFlip("horizontal_and_vertical"),
  RandomRotation(0.2),
])

In [34]:
from keras.applications.vgg16 import VGG16
from keras.applications import ResNet50
# base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model = ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)
base_model.trainable = False

classes = 2

model = tf.keras.models.Sequential([])

model.add(resize_and_rescale)
model.add(data_augmentation)
model.add(base_model)
model.add(Flatten())
# model.add(Dense(units=250, activation=relu))
model.add(Dense(units=classes, activation=linear, name='logits'))
model.add(Activation(activation=softmax))
# model.add(Dense(classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
epochs = 10

model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
)

Epoch 1/10
1/1 [==============================] - 33s 33s/step - loss: 0.8249 - accuracy: 0.5556 - val_loss: 11.4151 - val_accuracy: 0.4545
Epoch 2/10
1/1 [==============================] - 6s 6s/step - loss: 11.7519 - accuracy: 0.4444 - val_loss: 6.1531 - val_accuracy: 0.4545
Epoch 3/10
1/1 [==============================] - 5s 5s/step - loss: 6.3916 - accuracy: 0.4444 - val_loss: 1.7416 - val_accuracy: 0.5455
Epoch 4/10
1/1 [==============================] - 4s 4s/step - loss: 1.7291 - accuracy: 0.5556 - val_loss: 4.0327 - val_accuracy: 0.5455
Epoch 5/10
1/1 [==============================] - 4s 4s/step - loss: 3.9119 - accuracy: 0.5556 - val_loss: 3.3258 - val_accuracy: 0.5455
Epoch 6/10
1/1 [==============================] - 4s 4s/step - loss: 3.2471 - accuracy: 0.5556 - val_loss: 0.7971 - val_accuracy: 0.5455
Epoch 7/10
1/1 [==============================] - 4s 4s/step - loss: 0.8196 - accuracy: 0.5556 - val_loss: 3.0680 - val_accuracy: 0.4545
Epoch 8/10
1/1 [=====================

In [ ]:
from PIL import Image
Image.open('dog.png')